In [3]:
import sys

def change_python_version(pv):
    
    if "/lib" in pv:
        pv = pv.replace("/lib", "/envs/shoe_ml_evaluator/lib")
    
    if "python36" in pv:
        pv = pv.replace("python36", "python37")
    elif "python3.6" in pv:
        pv = pv.replace("python3.6", "python3.7")

    return pv

updated_path = list(map(change_python_version, sys.path))
sys.path.extend(updated_path)

In [4]:
import cv2
import glob
from joblib import dump,load
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
import webcolors

ModuleNotFoundError: No module named 'cv2.cv2'

In [ ]:
df = pd.read_csv("../data/browse_api_product_info.csv")

In [ ]:
df = df[["brand", "category", "colorway", "gender", "title", "averageDeadstockPrice", "deadstockSold", "imageUrl", "pricePremium"]]

In [ ]:
df

In [ ]:
def find_colours(colorway):
    
    colours = " ".join(colorway.lower().split("/")).split(" ")
    
    unique_colours = list(set([colour_str for colour_str in colours if colour_str in webcolors.CSS3_NAMES_TO_HEX]))
    
    return sorted(unique_colours)

df["uniqueColours"] = df["colorway"].apply(find_colours)

In [ ]:
df.head()

In [ ]:
# Entries where matching colour has not been found
df[df['uniqueColours'].map(lambda d: len(d)) == 0].shape

In [ ]:
mlb = MultiLabelBinarizer()
unique_colour_labels = mlb.fit_transform(df['uniqueColours'])
df = df.join(pd.DataFrame(unique_colour_labels, columns=mlb.classes_))
df.drop(["uniqueColours"], axis=1, inplace=True)
df.head()

In [ ]:
cols_to_rename = {col: f"colour_{col}" for col in df.columns[9:]}
df.rename(index=str, columns=cols_to_rename, inplace=True)
df.columns

In [ ]:
def encode_categorical_features(df, category):
    
    encoded = pd.get_dummies(df[category], prefix=category)
    
    df = df.join(encoded)
    df.drop([category], axis=1, inplace=True)
    return df

In [ ]:
for category in ["brand", "category", "gender"]:
    df = encode_categorical_features(df, category)

In [ ]:
df.head()

In [ ]:
cols = list(df.columns.values)
cols.pop(cols.index("deadstockSold"))
df = df[cols+["deadstockSold"]]

In [ ]:
df.head()

In [ ]:
df.drop(["colorway", "title", "imageUrl"], axis=1, inplace=True)

In [ ]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.loc[:,"averageDeadstockPrice"] = MinMaxScaler().fit_transform(X_train["averageDeadstockPrice"].values.reshape(-1, 1))
X_test.loc[:,"averageDeadstockPrice"] = MinMaxScaler().fit_transform(X_test["averageDeadstockPrice"].values.reshape(-1, 1))

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
df.columns.values

In [ ]:
# X_train.loc[:,"averageDeadstockPrice"] = MinMaxScaler().fit_transform(X_train["averageDeadstockPrice"].values.reshape(-1, 1))

In [ ]:
dump(df, "df.joblib")
dump(X_train, "X_train.joblib")
dump(y_train, "y_train.joblib")
dump(X_test, "X_test.joblib")
dump(y_test, "y_test.joblib")

In [ ]:
df.head()

In [ ]:
import seaborn as sns

%matplotlib inline

In [ ]:
sns.distplot(df["averageDeadstockPrice"], kde=False, rug=True)

In [ ]:
sns.distplot(df["averageDeadstockPrice"].apply(np.log), kde=False, rug=True)

In [ ]:
print("df[\"averageDeadstockPrice\"].min()", df["averageDeadstockPrice"].min())
print("df[\"averageDeadstockPrice\"].max()", df["averageDeadstockPrice"].max())

In [ ]:
sns.distplot(df["pricePremium"], kde=False, rug=True)

In [ ]:
sns.distplot(df["pricePremium"].apply(lambda p: np.log(p+1)), kde=False, rug=True)

In [ ]:
sns.distplot(df["deadstockSold"], kde=False, rug=True)

In [ ]:
sns.distplot(df["deadstockSold"].apply(lambda p: np.log(p+1)), kde=False, rug=True)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/browse_api_product_info.csv")

In [4]:
df.imageUrl

0      https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
1      https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
2      https://stockx.imgix.net/Adidas-Yeezy-Wave-Run...
3      https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
4      https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
5      https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
6      https://stockx.imgix.net/Adidas-Ultra-Boost-4-...
7      https://stockx.imgix.net/Air-Jordan-11-Retro-W...
8      https://stockx.imgix.net/Air-Jordan-1-Retro-Bl...
9      https://stockx.imgix.net/Revenge-X-Storm-Low-T...
10     https://stockx.imgix.net/Air-Jordan-11-Retro-M...
11     https://stockx.imgix.net/Adidas-Ultra-Boost-4p...
12     https://stockx.imgix.net/Nike-Air-Max-1-Annive...
13     https://stockx.imgix.net/Adidas-Yeezy-Boost-35...
14     https://stockx.imgix.net/Adidas-Ultra-Boost-4p...
15     https://stockx.imgix.net/Nike-Air-Max-97-Metal...
16     https://stockx.imgix.net/Nike-Air-Max-97-White...
17     https://stockx.imgix.net